## SVM+DECT

In [ ]:
import torch
from torch_geometric.data import Data, Batch
from sklearn.datasets   import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm       import SVC
from sklearn.metrics   import accuracy_score

# 1) import your weighted ECT
from wect import ECTLayer, ECTConfig

# 2) load + preprocess “Letters”  
X, y = fetch_openml('letter', version=1, return_X_y=True)
y = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

# 3) build PyG Data list, **including** all weight fields
def to_data_list(X, y):
    data_list = []
    for xi, yi in zip(X, y):
        xi = torch.tensor(xi, dtype=torch.float).unsqueeze(0)  # shape [1,16]
        data_list.append(Data(
            x            = xi,
            y            = torch.tensor([yi]),           # needed for batch.y
            node_weights = torch.ones(xi.size(0)),       # 1 weight per node
            edge_index   = torch.empty((2,0), dtype=torch.long),
            face         = torch.empty((3,0), dtype=torch.long),
            edge_weights = torch.empty((0,), dtype=torch.float),
            face_weights = torch.empty((0,), dtype=torch.float),
        ))
    return data_list

train_data = to_data_list(X_train, y_train)
test_data  = to_data_list(X_test,  y_test)

train_batch = Batch.from_data_list(train_data)
test_batch  = Batch.from_data_list(test_data)

# 4) set up weighted‑differentiable ECT  
config = ECTConfig(
    ect_type   = "points",
    bump_steps = 128,
    radius     = 1.0,
    normalized = False,
    fixed      = True
)
num_dims   = X_train.shape[1]   # 16 features
num_thetas = 128
# random projection directions in R^16:
v = torch.randn(num_dims, num_thetas)

ect_layer = ECTLayer(config, v=v)

# extract + flatten ECT features
def extract(batch):
    out = ect_layer(batch)              # shape [N, num_thetas, bump_steps]
    N   = batch.y.size(0)               # number of samples
    return out.detach().cpu().numpy().reshape(N, -1)

E_train = extract(train_batch)
E_test  = extract(test_batch)

# 5) train & evaluate SVM  
svm = SVC(kernel="rbf", C=1.0, gamma="scale")
svm.fit(E_train, y_train)
y_pred = svm.predict(E_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2%}")


## NN +DECT

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Generate random directions for ECT projection
def generate_random_directions(num_directions, dim):
    directions = torch.randn(dim, num_directions)
    directions = directions / torch.norm(directions, dim=0, keepdim=True)
    return directions

# ECT Layer
class ECTLayer(nn.Module):
    def __init__(self, v):
        super(ECTLayer, self).__init__()
        self.v = v  # Random directions

    def forward(self, batch):
        nh = batch.x @ self.v  # Project features onto directions
        return nh

# ECT Classifier
class ECTClassifier(nn.Module):
    def __init__(self, dim, num_directions, hidden_dim, num_classes):
        super(ECTClassifier, self).__init__()
        self.v = generate_random_directions(num_directions, dim)
        self.ect_layer = ECTLayer(self.v)
        self.fc1 = nn.Linear(num_directions, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, batch):
        ect_features = self.ect_layer(batch)
        ect_features = ect_features.view(-1, ect_features.size(1))
        x = F.relu(self.fc1(ect_features))
        x = self.fc2(x)
        return x

# Load and prepare data for Letters dataset
def load_letters_data():
    # Load the Letters dataset from OpenML
    data = fetch_openml(name='letter', version=1)
    X = data.data.to_numpy()  # Features (16-dimensional)
    y = data.target.to_numpy()  # Labels (A-Z)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    # Convert training data to PyTorch Geometric Data objects
    train_data = [Data(x=torch.tensor(X_train[i:i+1], dtype=torch.float32),
                       y=torch.tensor([ord(y_train[i]) - ord('A')], dtype=torch.long))
                  for i in range(len(X_train))]
    # Convert test data to PyTorch Geometric Data objects
    test_data = [Data(x=torch.tensor(X_test[i:i+1], dtype=torch.float32),
                      y=torch.tensor([ord(y_test[i]) - ord('A')], dtype=torch.long))
                 for i in range(len(X_test))]
    return train_data, test_data

# Training function
def train(model, train_data, num_epochs=200):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        model.train()
        batch = Batch.from_data_list(train_data)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, data_list):
    model.eval()
    batch = Batch.from_data_list(data_list)
    with torch.no_grad():
        out = model(batch)
        pred = out.argmax(dim=1)
        correct = (pred == batch.y).sum().item()
        accuracy = correct / len(batch.y)
    return accuracy

# Main execution
def main():
    # Parameters for Letters dataset
    dim = 16  # Feature dimension for Letters dataset
    num_directions = 128
    hidden_dim = 64
    num_classes = 26  # Number of classes (A-Z)
    num_epochs = 200

    # Load data
    train_data, test_data = load_letters_data()

    # Initialize and train model
    model = ECTClassifier(dim, num_directions, hidden_dim, num_classes)
    train(model, train_data, num_epochs)

    # Compute accuracies
    train_accuracy = evaluate(model, train_data)
    test_accuracy = evaluate(model, test_data)

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

if __name__ == "__main__":
    main()